In [4]:
import xml.sax
import timeit
import subprocess
import mwparserfromhell
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import Stemmer
import re
import math
invertedIndex={}
MAX_DOC_COUNT=100
index_ct=0
docid_wordcount={}
doc_id=0

reg1 = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
reg2 = re.compile(r'{\|(.*?)\|}',re.DOTALL)
reg3 = re.compile(r'{{v?cite(.*?)}}',re.DOTALL)
refreg=r'{{v?cite(.*?)}}'
reg4 = re.compile(r'[-.,:;_?()"/\']',re.DOTALL)
reg5 = re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL)
# reg6 = re.compile(r'[\'~` \n\"_!=@#$%-^*+{\[}\]\|\\<>/?]',re.DOTALL)
reg6 = re.compile(r'[\'~` \n\"_!=@#$%\-^*+{\[}\]\|\\<>/?]',re.DOTALL)
catRegExp = r'\[\[category:(.*?)\]\]'
infoRegExp = r'{{infobox(.*?)}}'
refRegExp = r'== ?references ?==(.*?)=='
reg7 = re.compile(infoRegExp,re.DOTALL)
reg8 = re.compile(refRegExp,re.DOTALL)
reg9 = re.compile(r'{{(.*?)}}',re.DOTALL)
reg10 = re.compile(r'<(.*?)>',re.DOTALL)
reg11=re.compile(r'\[\[(.*?)\]\]')
reg12=re.compile(catRegExp,re.DOTALL)
stop_words = set(stopwords.words('english'))
stemmer = Stemmer.Stemmer('english') #pystemmer used to stem words
invertedIndex={}



class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []
        self._id=1
        self._doc_ct=1
        self._raw_token=0
#         self._currentPage

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

            parsePage(self,self._values,self._doc_ct)
            self._pages = []
            self._id=self._id+1
#             self._doc_ct+=1
#             if self._doc_ct==5000:
#                 doc_ct=1

def remove_newline(text):
    text=text.replace('\n',' ')
    text=text.strip()
    return text

def process_digit(w):
    if w.isnumeric():
        w=w.lstrip('0')
    return w

def cleantext(tokenlist):
    tokentext=' '.join(tokenlist)
#     print(tokentext)
    tokentext=reg4.sub(' ',tokentext)
    tokentext=reg5.sub(' ',tokentext)
    tokentext=reg6.sub(' ',tokentext)
    tokenlist=tokentext.split()
    return tokenlist
    
# def is_valid(word):
#     if word.isalnum() and len(word)>=2:
#         return True
#     else:
#         return False
    
def is_valid(word):
	if word == ""  or len(word) < 3:
			return False
	try:
		word.encode(encoding='utf-8').decode('ascii')
	except UnicodeDecodeError:
		return False
	else:
		return True
    
    
def field_word_count(dictionary):
    ct=0
    for key,val in dictionary.items():
        ct+=val
    return ct
        
    

#inserting into global index
# global index is a dict of dict of dict
#first level dict 
def insertIntoIndex(dictionary,field,doc_id):
    
    word_ct=field_word_count(dictionary)
    for key,val in dictionary.items():
        if key not in invertedIndex:
            invertedIndex[key]={}
        if doc_id not in invertedIndex[key]:
            invertedIndex[key][doc_id]={}
            
        tf=val/word_ct
        tf=round(math.log(1+tf,10),4)
        
        if field=="title":                      #precedence order  title>infobox=category>ref=external link>body
            tf=int((.0100+tf)*10000)            # hence weight of title's tfidf is increased followed by others
        if field=="infobox":
            tf=int((.0050+tf)*10000)
        if field=="category":
            tf=int((.0050+tf)*10000)
        if field=="ref":
            tf=int((.0010+tf)*10000)
        if field=="el":
            tf=int((.0010+tf)*10000)
        if field=="body":
            tf=int((.0001+tf)*10000)
        
        tf_val=[tf,val]
        invertedIndex[key][doc_id][field]=tf_val
#         invertedIndex[key][doc_id][field]=val



# dump global dict content into intermediate inverted_index which later will be merged using merge function
def dumpindex():
    global index_ct,docid_wordcount,invertedIndex
    print("dump",index_ct)
    invertedIndexFile=open("inverted_index/"+str(index_ct)+".txt",'w')
    index_ct=index_ct+1
    words=sorted(invertedIndex.keys())
    clean_tokens=len(invertedIndex.keys())
    
#     content=""
    
    for word in words:
        content=""+word
        
        documentIdDict=invertedIndex[word]
        
        for doc_id in sorted(documentIdDict.keys()):
            content=content+"#"+"id"+str(doc_id)
            word_ct=0
            
            if "body" in invertedIndex[word][doc_id]:
                content=content+"b"+str(invertedIndex[word][doc_id]["body"][0])
                word_ct=word_ct+invertedIndex[word][doc_id]["body"][1]
            
            if "title" in invertedIndex[word][doc_id]:
                content=content+"t"+str(invertedIndex[word][doc_id]["title"][0])
                word_ct=word_ct+invertedIndex[word][doc_id]["title"][1]
                
            if "category" in invertedIndex[word][doc_id]:
                content=content+"c"+str(invertedIndex[word][doc_id]["category"][0])
                word_ct=word_ct+invertedIndex[word][doc_id]["category"][1]
                
            if "infobox" in invertedIndex[word][doc_id]:
                content=content+"i"+str(invertedIndex[word][doc_id]["infobox"][0])
                word_ct=word_ct+invertedIndex[word][doc_id]["infobox"][1]
                
            if "el" in invertedIndex[word][doc_id]:
                content=content+"e"+str(invertedIndex[word][doc_id]["el"][0])
                word_ct=word_ct+invertedIndex[word][doc_id]["el"][1]
                
            if "ref" in invertedIndex[word][doc_id]:
                content=content+"r"+str(invertedIndex[word][doc_id]["ref"][0])
                word_ct=word_ct+invertedIndex[word][doc_id]["ref"][1]
#             print(word_ct,docid_wordcount)
            tf = word_ct/docid_wordcount[doc_id]
            tf = round(math.log(1 + tf, 10), 4)
            tf = int((.0001+tf)*10000)
            content=content+"@"+str(tf)

            
                
                
        invertedIndexFile.write(content+'\n')
    invertedIndexFile.close()
    invertedIndex={}
    return clean_tokens
        

    
    

    

    

# wikipedia page will be parsed here
def parsePage(self,page,doc_ct):
    global docid_wordcount,invertedIndex
    global doc_id
    
    word_ct=0


    title=page['title']
#     title=title.lower()
    body=page['text']
    body=reg1.sub(' ',body)
    body=reg2.sub(' ',body)
#     body=reg3.sub(' ',body)   process reference first and then place it
    body=reg10.sub(' ',body)
    body=remove_newline(body)
    body=body.lower()
    
    infodict={}
    textdict={}
    catdict={}
    titledict={}
    externalLinkDict={}
    refdict={}
#     print(doc_id)
    

        
#     processing references
    
#     if doc_id==12:
    references=re.findall(refreg,body,re.DOTALL)


    for ref in references:
        reftitle=re.findall(r'\| ?title ?=(.*?)\|',ref,re.DOTALL) #title is a list here

        if(len(reftitle)!=0):
            reftitle=reftitle[0]
        else:
            reftitle=""

        reftitletokens=reftitle.split()
        reftitletokens=cleantext(reftitletokens)
        self._raw_token=self._raw_token+len(reftitletokens)
        reftitletokens=[word for word in reftitletokens if word not in stop_words]
        reftitletokens=stemmer.stemWords(reftitletokens)
        for token in reftitletokens:
            if is_valid(token):
                token=process_digit(token)
                word_ct+=1
                if token not in refdict:
                    refdict[token]=1
                else:
                    refdict[token]+=1

        
    body=reg3.sub(' ',body) #all {{cite}} are already processed so {{cite}} no more required.
            
            
        
    
    
#     processing infobox
    

    infobox=re.findall(infoRegExp,body,re.DOTALL)


    for info in infobox:
        infoTitle=re.findall(r'^(.*?)\|',info,re.DOTALL)
        infotokens=re.findall(r'=(.*?)\|',info,re.DOTALL)
        if len(infoTitle)!=0:
            infotokens.append(infoTitle[0])
        
        infotokens=cleantext(infotokens)
        self._raw_token=self._raw_token+len(infotokens)


        infotokens=[word for word in infotokens if word not in stop_words]
        infotokens=stemmer.stemWords(infotokens)
        for token in infotokens:
            if is_valid(token):
                token=process_digit(token)
                word_ct+=1
                if token not in infodict:
                    infodict[token]=1;
                else:
                    infodict[token]+=1;
                    
# processing category
    

    category=re.findall(catRegExp,body,re.DOTALL)
    category=cleantext(category)
    self._raw_token=self._raw_token+len(category)
    category=[word for word in category if word not in stop_words]
    category=stemmer.stemWords(category)
    for token in category:
        if is_valid(token):
            token=process_digit(token)
            word_ct+=1
            if token not in catdict:
                catdict[token]=1
            else:
                catdict[token]+=1
                
# processing title
    
        
    title=re.findall(r'[A-Z][a-z]*',title,re.DOTALL)
    for i in range(len(title)):
        title[i]=title[i].lower()
    
    self._raw_token=self._raw_token+len(title)
    title=stemmer.stemWords(title)

    
    for token in title:
        if is_valid(token):
            token=process_digit(token)
            word_ct+=1
            if token not in titledict:
                titledict[token.lower()]=1
            else:
                titledict[token.lower()]+=1;

            
#processing external link     
    linkcontent=re.findall(r'== ?external links ?==.*',body,re.DOTALL)  #findall returns a list

    if len(linkcontent)!=0:
        linkcontent=reg11.sub(' ',linkcontent[0])
        link=re.findall(r'\[(.*?)\]', linkcontent, flags=re.MULTILINE)
        link=cleantext(link)
        self._raw_token=self._raw_token+len(link)
        link=[word for word in link if word not in stop_words]
        link=stemmer.stemWords(link)
        for l in link:
            if is_valid(l):
                l=process_digit(l)
                word_ct+=1
                if l not in externalLinkDict:
                    externalLinkDict[l]=1
                else:
                     externalLinkDict[l]+=1
                    
    
    
#processing text box
                    
    body=reg4.sub(' ',body)
    body=reg5.sub(' ',body)
    body=reg6.sub(' ',body)
    body=reg7.sub(' ',body)
    body=reg9.sub(' ',body)
    body=reg11.sub(' ',body)
    body=reg12.sub(' ',body)
    
   
    bodytokens=cleantext(body.split())
    self._raw_token=self._raw_token+len(bodytokens)

    bodytokens=[word for word in bodytokens if word not in stop_words]
    bodytokens=stemmer.stemWords(bodytokens)

    for token in bodytokens:
        if is_valid(token):
            token=process_digit(token)
            word_ct+=1
            if token not in textdict:
                textdict[token]=1
            else:
                textdict[token]+=1
                
                
    docid_wordcount[doc_id]=word_ct

    insertIntoIndex(infodict,"infobox",doc_id)
    insertIntoIndex(catdict,"category",doc_id)
    insertIntoIndex(titledict,"title",doc_id)
    insertIntoIndex(externalLinkDict,"el",doc_id)
    insertIntoIndex(textdict,"body",doc_id)
    insertIntoIndex(refdict,"ref",doc_id)
    
    
    if(self._doc_ct==30000):
#         print("dump_______________________________----",doc_id)
        self._doc_ct=0
        dumpindex()
        invertedIndex={}
        
    self._doc_ct=self._doc_ct+1
    doc_id=doc_id+1
    
        
        
    

In [5]:
import os
import heapq
import math


def field_idf_for_word(doc_list):
    total_doc=8796395
    t_ct=1
    c_ct=1
    i_ct=1
    r_ct=1
    e_ct=1
    b_ct=1
    for doc in doc_list:
        posting_t=re.findall(r't[0-9]*',doc,re.DOTALL)
        posting_c=re.findall(r'c[0-9]*',doc,re.DOTALL)
        posting_i=re.findall(r'i[0-9]+',doc,re.DOTALL)   #otherwise it might match with id123
        posting_r=re.findall(r'r[0-9]*',doc,re.DOTALL)
        posting_e=re.findall(r'e[0-9]*',doc,re.DOTALL)
        posting_b=re.findall(r'b[0-9]*',doc,re.DOTALL)
        
        if(len(posting_t)):
            t_ct+=1
        if(len(posting_c)):
            c_ct+=1
        if(len(posting_i)):
            i_ct+=1
        if(len(posting_r)):
            r_ct+=1
        if(len(posting_e)):
            e_ct+=1
        if(len(posting_b)):
            b_ct+=1
    
    idf_t=total_doc/t_ct
    idf_t=round(math.log(1+idf_t,10),3)
    
    idf_c=total_doc/c_ct
    idf_c=round(math.log(1+idf_c,10),3)
    
    idf_i=total_doc/i_ct
    idf_i=round(math.log(1+idf_i,10),3)
    
    idf_r=total_doc/r_ct
    idf_r=round(math.log(1+idf_r,10),3)
    
    idf_e=total_doc/e_ct
    idf_e=round(math.log(1+idf_e,10),3)
    
    idf_b=total_doc/b_ct
    idf_b=round(math.log(1+idf_b,10),3)
    
    idf_res=[idf_t,idf_c,idf_i,idf_r,idf_e,idf_b]
    
    return idf_res


def tfidf_posting(tf_posting,field_idf_list):
    tfidf_pos=""
    
    
    docid=re.findall(r'id[0-9]*',tf_posting,re.DOTALL)
#     docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
    tfidf_pos+=docid[0]       # doc id added
    
    t_pos=re.findall(r't[0-9]*',tf_posting,re.DOTALL)
    if(len(t_pos)):
        t_tf=int(re.findall(r'[0-9]+',t_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(t_tf*field_idf_list[0]))
        tfidf_pos+="t"+str(tfidf)
        
    c_pos=re.findall(r'c[0-9]*',tf_posting,re.DOTALL)
    if(len(c_pos)):
        c_tf=int(re.findall(r'[0-9]+',c_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(c_tf*field_idf_list[1]))
        tfidf_pos+="c"+str(tfidf)
        
    i_pos=re.findall(r'i[0-9]+',tf_posting,re.DOTALL)
    if(len(i_pos)):
        i_tf=int(re.findall(r'[0-9]+',i_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(i_tf*field_idf_list[2]))
        tfidf_pos+="i"+str(tfidf)
        
    r_pos=re.findall(r'r[0-9]*',tf_posting,re.DOTALL)
    if(len(r_pos)):
        r_tf=int(re.findall(r'[0-9]+',r_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(r_tf*field_idf_list[3]))
        tfidf_pos+="r"+str(tfidf)
    
    e_pos=re.findall(r'e[0-9]*',tf_posting,re.DOTALL)
    if(len(e_pos)):
        e_tf=int(re.findall(r'[0-9]+',e_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(e_tf*field_idf_list[4]))
        tfidf_pos+="e"+str(tfidf)
    
    b_pos=re.findall(r'b[0-9]*',tf_posting,re.DOTALL)
    if(len(b_pos)):
        b_tf=int(re.findall(r'[0-9]+',b_pos[0],re.DOTALL)[0])
        tfidf=int(math.floor(b_tf*field_idf_list[5]))
        tfidf_pos+="b"+str(tfidf)
        
        
    return tfidf_pos
        
    
    
    
    
    
    
    
    
        
            
        
        
        
        
def merge(index_path,index_count):    #merge seperatly build index file
    print(index_count," index_count")
    word_limit=0
    word_limit_max=40000
    final_index_count=0
    total_docs=8796395
    token_count=0
#     total_docs=2
    
    merged_index=index_path+"/merged_path"
    
    if not os.path.exists(merged_index):
        os.makedirs(merged_index)
        
    final_fp=open(merged_index+"/"+str(final_index_count)+".txt",'w')
    secondary_idx = open( merged_index+ "/" + "secondary_index.txt",'w')
        
    heap_list=[]
    file_pointers=[]
    
    i=0
    while(i<index_count):
#         print(i,"file pointer update")
        file_pointers.append(open(index_path+"/"+str(i)+".txt"))
        i+=1
        
    i=0 #index file no from which word is considered
    while(i<index_count):
#         print(i,"update heap")
        line=file_pointers[i].readline()
        word=line[:line.index('#')]
        posting=line[line.index('#')+1:]
        tpl=(word,i,posting)
        heap_list.append(tpl)
        token_count+=1
        i+=1
        
#     print()
        
    heapq.heapify(heap_list)
    
    pre_touple=heapq.heappop(heap_list)
    pre_word=pre_touple[0]
    pre_idx=pre_touple[1]
    pre_posting=pre_touple[2]
    pre_posting[:-1]
    line=file_pointers[pre_idx].readline()
    if line!="":
        word=line[:line.index('#')]
        print()
        posting=line[line.index('#')+1:]
        tpl=(word,pre_idx,posting)
        heap_list.append(tpl)
        heapq.heapify(heap_list)
        
    
    while(len(heap_list)>0):           #merge using a heap
        token_count+=1
        


        
        cur_tpl=heapq.heappop(heap_list)

        line=file_pointers[cur_tpl[1]].readline()
        
        if line!="":

            word=line[:line.index('#')]

            posting=line[line.index('#')+1:]
            tpl=(word,cur_tpl[1],posting)
            heap_list.append(tpl)
            heapq.heapify(heap_list)
        
        if(pre_word==cur_tpl[0]):                    #note that no writing happning here so no need to convert tf posting to tfidf posting
            pre_idx=cur_tpl[1]
          
            pre_posting=pre_posting+"#"+cur_tpl[2]
            pre_posting=pre_posting[:-1]
        else:                                       #working on this
            docs=pre_posting.split("#")
            

            idf = total_docs/len(docs)              #overall idf
            idf = round(math.log(1 + idf, 10), 3)
            
            line_to_write=pre_word                  #word to be written 
            
            field_idf_list=field_idf_for_word(docs)   # get field tfidf list here index 0=t,1=c,2=i,3=r,4=e,5=b
            

    
            
            for doc in docs:                        #modifying each doc's field tf to tfidf and writing to merged file
                info=doc.split("@")
                tf=int(info[1])
                tfidf = int(math.floor(tf*idf))    #overall tfidf of word in some document
                
                
                
                
                tf_posting_section=info[0]
                tfidf_posting_section= tfidf_posting(tf_posting_section,field_idf_list) #convert tf posting  part to tfidf posting part
                
                


#                 line_to_write+="#"+info[0]+"@"+str(tfidf)  #v.0
                line_to_write+="#"+tfidf_posting_section+"@"+str(tfidf) #v.1
                
        
        
    
                
            if(line_to_write!=pre_word):
                final_fp.write(line_to_write+'\n')
                
            if word_limit==0:   #first time entry after write happpen so write the first word into secondary index, each merged file is having a word limit
                secondary_idx.write(pre_word+'\n')
                
            word_limit+=1

                
            if word_limit==word_limit_max: # if word limit reach word_limit_max then open new file

                word_limit=0
                final_index_count+=1
                final_fp=open(merged_index+"/"+str(final_index_count)+".txt",'w')
                
            pre_touple=cur_tpl
            pre_word=pre_touple[0]
            pre_idx=pre_touple[1]
#             print(pre_idx,"pre idx else")
            pre_posting=pre_touple[2]
            pre_posting=pre_posting[:-1]
        
    return token_count
        

            
#             if(len(heap_list)==0):
                
        
            
#     del_index(index_path,index_count)   #if you want to delete intermediate inverted index then uncomment this
    
                

        
        

def del_index(index_path,index_file_ct):     
    i=0
    while i<index_file_ct:
        file_delete=index_path+"/"+str(i)+".txt"
        if os.path.exists(file_delete):
            os.remove(file_delete)
        i=i+1
        
        
        
        

    

In [6]:


# import index_creation
import xml.sax
import timeit
import subprocess
import mwparserfromhell
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import Stemmer
import re
import sys
from pathlib import Path
import os
import math
from os import listdir
from os.path import isfile, join



def main():
    start = timeit.default_timer()
    global doc_ct,index_ct
    
#     data_path=sys.argv[1]
#     data_path="wiki_dump/enwiki-20200801-pages-articles-multistream1.xml-p1p30303.bz2"
    data_directory="wiki_dump/"
    data_paths = [f for f in listdir(data_directory) if isfile(join(data_directory, f))]
    data_paths.sort()
    
    
    #data_paths is a list of names of all 34 wikidump file
    

    for data_path in data_paths:

        print(data_path)

        handler = WikiXmlHandler()
        parser = xml.sax.make_parser()
        parser.setContentHandler(handler)
        doc_ct=0
        for line in subprocess.Popen(['bzcat'], 
                                  stdin = open(data_directory+data_path), 
                                  stdout = subprocess.PIPE).stdout:
            parser.feed(line)
            
        
#             if len(handler._pages) > 1:
#                 break
        if(len(invertedIndex)):
            dumpindex() 
        print("documents processed",doc_id)


    print("final dump")
    if(len(invertedIndex)):
        dumpindex()
    stat_path="stat.txt"

    stop = timeit.default_timer()
    stat_fp=open(stat_path,'w')
    
    token_count=merge("inverted_index",index_ct)   

    
    print("token count",token_count)
    print("time taken:",(stop-start)/60)
    print("total docs",doc_id)


if __name__ == "__main__": 
	main() 

enwiki-20200801-pages-articles-multistream1.xml-p1p30303.bz2
dump 0
documents processed 19797
enwiki-20200801-pages-articles-multistream10.xml-p2336423p3046512.bz2
dump 1
dump 2
dump 3
dump 4
dump 5
dump 6
dump 7
dump 8
dump 9
dump 10
dump 11
documents processed 328244
enwiki-20200801-pages-articles-multistream11.xml-p3046513p3926861.bz2
dump 12
dump 13
dump 14
dump 15
dump 16
dump 17
dump 18
dump 19
dump 20
dump 21
dump 22
dump 23
documents processed 664651
enwiki-20200801-pages-articles-multistream12.xml-p3926862p5040436.bz2.part
dump 24
dump 25
dump 26
dump 27
dump 28
dump 29
dump 30
documents processed 863751
enwiki-20200801-pages-articles-multistream13.xml-p5040437p6197594.bz2.part
dump 31
dump 32
dump 33
dump 34
dump 35
dump 36
dump 37
dump 38
documents processed 1089351
enwiki-20200801-pages-articles-multistream14.xml-p6197595p7697594.bz2.part
dump 39
dump 40
dump 41
dump 42
dump 43
dump 44
dump 45
dump 46
documents processed 1313251
enwiki-20200801-pages-articles-multistream14.